In [1]:
import pickle
import warnings
import copy
import pandas as pd

In [2]:
num_groups = 2 # number of groups per matching group
num_match_grp = 6
num_rounds = 6

In [3]:
class Groups:
    def __init__(self, num_players, num_groups):
        self.was_first = [False] * num_players
        self.num_groups = num_groups
        self.results = []

    def add_round(self, allocation):
        is_trial_round = len(self.results) == 0
        self.results.append([[] for _ in range(self.num_groups)])
        for player, group in enumerate(allocation):
            group_id = int(group.lstrip("Group ")) - 1
            if self.was_first[player]:
                self.results[-1][group_id].append(player)
            else:
                self.results[-1][group_id].insert(0, player)
        for group in range(len(self.results[-1])):
            first_in_group = self.results[-1][group][0]
            if self.was_first[self.results[-1][group][0]]:
                warnings.warn(f"Player {first_in_group} was first multiple times")
            self.was_first[first_in_group] = True if not is_trial_round else False

    def adjust_indices(self):
        for round in range(len(self.results)):
            for group in range(len(self.results[round])):
                for player in range(len(self.results[round][group])):
                    self.results[round][group][player] = (
                        self.results[round][group][player] + 1
                    )

In [4]:
golfers = pd.read_csv(
    "golfer_solution.csv", index_col=0
)
golfers.index = [int(name.lstrip("Player ")) - 1 for name in golfers.index]
golfers.columns = [int(name.lstrip("Round ")) - 1 for name in golfers.columns]

groups = Groups(len(golfers), num_groups)
for round in golfers.columns:
    groups.add_round(golfers.loc[:, round])
groups.adjust_indices()

/var/folders/ml/b6fb3hq91m70wjn5hjgyky58kdcssc/T/ipykernel_18948/62703243.py:19: UserWarning: Player 3 was first multiple times
  warnings.warn(f"Player {first_in_group} was first multiple times")
/var/folders/ml/b6fb3hq91m70wjn5hjgyky58kdcssc/T/ipykernel_18948/62703243.py:19: UserWarning: Player 1 was first multiple times
  warnings.warn(f"Player {first_in_group} was first multiple times")
/var/folders/ml/b6fb3hq91m70wjn5hjgyky58kdcssc/T/ipykernel_18948/62703243.py:19: UserWarning: Player 0 was first multiple times
  warnings.warn(f"Player {first_in_group} was first multiple times")
/var/folders/ml/b6fb3hq91m70wjn5hjgyky58kdcssc/T/ipykernel_18948/62703243.py:19: UserWarning: Player 2 was first multiple times
  warnings.warn(f"Player {first_in_group} was first multiple times")


In [5]:
groups.results

[[[5, 3, 1], [6, 4, 2]],
 [[6, 5, 1], [4, 3, 2]],
 [[5, 3, 4], [2, 1, 6]],
 [[4, 5, 6], [3, 1, 2]],
 [[2, 3, 6], [1, 4, 5]],
 [[1, 2, 4], [3, 5, 6]]]

In [6]:
group_matrices = copy.deepcopy(groups.results)
for round_num in range(0, num_rounds): 
    for match_grp_num in range(1, num_match_grp):
        for i in range(num_groups): 
            group_matrices[round_num].append([pl_num + 6 * match_grp_num for pl_num in groups.results[round_num][i]])
group_matrices

[[[5, 3, 1],
  [6, 4, 2],
  [11, 9, 7],
  [12, 10, 8],
  [17, 15, 13],
  [18, 16, 14],
  [23, 21, 19],
  [24, 22, 20],
  [29, 27, 25],
  [30, 28, 26],
  [35, 33, 31],
  [36, 34, 32]],
 [[6, 5, 1],
  [4, 3, 2],
  [12, 11, 7],
  [10, 9, 8],
  [18, 17, 13],
  [16, 15, 14],
  [24, 23, 19],
  [22, 21, 20],
  [30, 29, 25],
  [28, 27, 26],
  [36, 35, 31],
  [34, 33, 32]],
 [[5, 3, 4],
  [2, 1, 6],
  [11, 9, 10],
  [8, 7, 12],
  [17, 15, 16],
  [14, 13, 18],
  [23, 21, 22],
  [20, 19, 24],
  [29, 27, 28],
  [26, 25, 30],
  [35, 33, 34],
  [32, 31, 36]],
 [[4, 5, 6],
  [3, 1, 2],
  [10, 11, 12],
  [9, 7, 8],
  [16, 17, 18],
  [15, 13, 14],
  [22, 23, 24],
  [21, 19, 20],
  [28, 29, 30],
  [27, 25, 26],
  [34, 35, 36],
  [33, 31, 32]],
 [[2, 3, 6],
  [1, 4, 5],
  [8, 9, 12],
  [7, 10, 11],
  [14, 15, 18],
  [13, 16, 17],
  [20, 21, 24],
  [19, 22, 23],
  [26, 27, 30],
  [25, 28, 29],
  [32, 33, 36],
  [31, 34, 35]],
 [[1, 2, 4],
  [3, 5, 6],
  [7, 8, 10],
  [9, 11, 12],
  [13, 14, 16],
  [15, 17

In [7]:
with open("group_matrices.pkl", "wb") as f:
    pickle.dump(group_matrices, f)